# Morphosyntactic Constraint Audit

**Tests category-level constraint at diagnostic syntactic cues**

## Research Question
Does syntactic structure shift probability mass toward grammatically-appropriate lexical classes (VERB/NOUN/FUNCTION) at diagnostic cue positions?

## Key Improvement over POS Audit
- **No POS tagger** → Uses lexicon-based classification
- **Word-start only** → Avoids BPE fragment issues
- **Multiple cue types** → Tests verb slots, noun slots, NP starts

## Cues Tested
1. **Infinitival "to"** → Expects VERB
2. **Modals** (can, will, would, etc.) → Expects VERB
3. **Aux/copula** (is, are, was, were) → Expects open-class
4. **Prepositions** (in, on, at, etc.) → Expects DET/NOUN

## Expected Pattern
- **Sentence & Jabberwocky**: High mass on expected class (structure intact)
- **Scrambled**: Lower mass (structure disrupted)

## Step 1: Install Dependencies

In [ ]:
!pip install -q transformers torch numpy

## Step 2: Upload Stimuli File

**IMPORTANT:** Click the **folder icon** (📁) on the left sidebar, then drag and drop `stimuli_with_scrambled.json` into the Files area.

Or run this cell:

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload stimuli_with_scrambled.json

## Step 3: Define Lexicons

In [ ]:
# Function words (closed-class, ~200 items)
FUNCTION_SET = {
    # Determiners
    'the', 'a', 'an', 'this', 'that', 'these', 'those', 'my', 'your', 'his',
    'her', 'its', 'our', 'their', 'some', 'any', 'no', 'every', 'each', 'either',
    'neither', 'much', 'many', 'more', 'most', 'few', 'several', 'all', 'both',
    # Pronouns
    'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them',
    'myself', 'yourself', 'himself', 'herself', 'itself', 'ourselves', 'themselves',
    'who', 'whom', 'whose', 'what', 'which', 'whoever', 'whomever', 'whatever',
    # Auxiliaries
    'is', 'are', 'am', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'having',
    'do', 'does', 'did', 'doing',
    'can', 'could', 'may', 'might', 'must', 'shall', 'should', 'will', 'would',
    # Prepositions
    'in', 'on', 'at', 'to', 'for', 'with', 'from', 'by', 'about', 'as', 'of',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'between',
    'among', 'under', 'over', 'against', 'within', 'without', 'throughout',
    # Conjunctions
    'and', 'or', 'but', 'nor', 'so', 'yet',
    'because', 'since', 'unless', 'until', 'while', 'although', 'though',
    'if', 'when', 'where', 'whether', 'than',
    # Negation & other
    'not', "n't", 'there', 'here', 'then', 'now', 'very', 'too', 'also', 'just', 'only',
}

# Common verbs
VERB_SET = {
    'be', 'have', 'do', 'say', 'get', 'make', 'go', 'know', 'take', 'see',
    'come', 'think', 'look', 'want', 'give', 'use', 'find', 'tell', 'ask',
    'work', 'seem', 'feel', 'try', 'leave', 'call', 'become', 'run', 'move',
    'live', 'believe', 'bring', 'happen', 'write', 'sit', 'stand', 'lose',
    'pay', 'meet', 'include', 'continue', 'set', 'learn', 'change', 'lead',
    'understand', 'watch', 'follow', 'stop', 'create', 'speak', 'read', 'allow',
    'add', 'spend', 'grow', 'open', 'walk', 'win', 'offer', 'remember', 'love',
    'consider', 'appear', 'buy', 'wait', 'serve', 'die', 'send', 'expect',
    'build', 'stay', 'fall', 'cut', 'reach', 'kill', 'raise', 'pass', 'sell',
    # -ing/-ed forms
    'going', 'making', 'doing', 'saying', 'getting', 'taking', 'seeing', 'coming',
    'looking', 'working', 'trying', 'running', 'playing', 'showing', 'talking',
    'went', 'made', 'said', 'got', 'took', 'saw', 'came', 'looked', 'worked',
}

# Common nouns
NOUN_SET = {
    'time', 'person', 'year', 'way', 'day', 'thing', 'man', 'world', 'life',
    'hand', 'part', 'child', 'eye', 'woman', 'place', 'work', 'week', 'case',
    'point', 'government', 'company', 'number', 'group', 'problem', 'fact',
    'people', 'water', 'room', 'mother', 'area', 'money', 'story', 'family',
    'student', 'word', 'business', 'country', 'question', 'school', 'state',
    'night', 'head', 'home', 'office', 'power', 'hour', 'game', 'line', 'end',
    'member', 'law', 'car', 'city', 'name', 'team', 'minute', 'idea', 'body',
    'dog', 'house', 'president', 'book', 'community', 'computer',
}

print(f"✓ Lexicons loaded:")
print(f"  FUNCTION: {len(FUNCTION_SET)} words")
print(f"  VERB:     {len(VERB_SET)} words")
print(f"  NOUN:     {len(NOUN_SET)} words")

## Step 4: Define Cue Specifications

In [ ]:
CUE_SPECS = {
    'infinitival_to': {
        'cues': ['to'],
        'expected_class': 'verb',
        'description': 'Infinitival "to" expects VERB'
    },
    'modal': {
        'cues': ['can', 'will', 'would', 'could', 'should', 'must', 'may', 'might'],
        'expected_class': 'verb',
        'description': 'Modals expect VERB'
    },
    'aux_copula': {
        'cues': ['is', 'are', 'was', 'were'],
        'expected_class': 'open_class',
        'description': 'Aux/copula expects open-class'
    },
    'preposition': {
        'cues': ['in', 'on', 'at', 'with', 'from', 'for'],
        'expected_class': 'function_or_noun',
        'description': 'Prepositions expect DET/NOUN'
    },
}

for cue_type, spec in CUE_SPECS.items():
    print(f"  {cue_type:20s}: {spec['description']}")

## Step 5: Classification Functions

In [ ]:
def is_word_start_token(token_str):
    """Check if BPE token starts a new word (space + letter)."""
    if not token_str:
        return False
    if token_str[0] == ' ' and len(token_str) > 1 and token_str[1].isalpha():
        return True
    return False

def decode_to_word(token_str):
    """Strip leading space and trailing punctuation to get base word."""
    word = token_str.strip()
    # Strip trailing punctuation
    while word and not word[-1].isalnum():
        word = word[:-1]
    return word.lower()

def classify_candidate(word):
    """
    Classify word into lexical class.
    Priority: FUNCTION > VERB > NOUN > OTHER
    """
    if not word:
        return 'other'

    # Check if punctuation only
    if all(not c.isalnum() for c in word):
        return 'punct'

    word_lower = word.lower()

    # Priority order
    if word_lower in FUNCTION_SET:
        return 'function'
    if word_lower in VERB_SET:
        return 'verb'
    if word_lower in NOUN_SET:
        return 'noun'

    return 'other_open'

print("✓ Classification functions defined")

## Step 6: Sanity Checks

In [ ]:
print("=" * 80)
print("SANITY CHECKS: Classification")
print("=" * 80)
print()

test_cases = [
    (' the', 'function', 'Determiner → FUNCTION'),
    (' and', 'function', 'Conjunction → FUNCTION'),
    (' is', 'function', 'Auxiliary → FUNCTION'),
    (' run', 'verb', 'Common verb → VERB'),
    (' running', 'verb', 'Verb -ing → VERB'),
    (' time', 'noun', 'Common noun → NOUN'),
    (' people', 'noun', 'Common noun → NOUN'),
    (',', 'punct', 'Comma → PUNCT (after decode)'),
    ('.', 'punct', 'Period → PUNCT (after decode)'),
]

all_passed = True
for token, expected, desc in test_cases:
    word = decode_to_word(token)
    result = classify_candidate(word)
    status = "✓" if result == expected else "✗ FAIL"
    if result != expected:
        all_passed = False
    print(f"{status}  {repr(token):15s} → {word:12s} → {result:12s}  ({desc})")

print()
if all_passed:
    print("✓ All sanity checks passed!")
else:
    print("✗ Some tests FAILED - check classification logic")
print()

## Step 7: Load Model and Stimuli

In [ ]:
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from collections import defaultdict

print("Loading GPT-2...")
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.eval()
print("✓ Model loaded\n")

print("Loading stimuli...")
with open('stimuli_with_scrambled.json') as f:
    stimuli = json.load(f)
print(f"✓ Loaded {len(stimuli)} stimulus sets\n")

## Step 8: Define Analysis Function

In [ ]:
def analyze_cue_position(model, tokenizer, text, cue_word, top_k=5000, debug=False):
    """
    Analyze predictions after a diagnostic cue.

    Returns:
        dict with:
        - mass: probability mass by class
        - top_candidates: top-20 word-start candidates (if debug=True)
    """
    words = text.split()

    # Find cue position
    cue_position = None
    for i, word in enumerate(words[:-1]):
        if word.lower().strip('.,!?;:') == cue_word.lower():
            cue_position = i
            break

    if cue_position is None:
        return None

    # Get context (prefix ending after cue)
    context = ' '.join(words[:cue_position+1])
    inputs = tokenizer(context, return_tensors='pt')

    # Get next-token predictions
    with torch.no_grad():
        outputs = model(**inputs)
        next_token_logits = outputs.logits[0, -1, :]
        probs = torch.softmax(next_token_logits, dim=-1)
        top_k_probs, top_k_ids = torch.topk(probs, min(top_k, len(probs)))

    # Classify candidates and accumulate mass
    mass = {
        'verb': 0.0,
        'noun': 0.0,
        'function': 0.0,
        'other_open': 0.0,
        'punct': 0.0,
        'non_wordstart': 0.0,
    }

    word_start_candidates = []

    for prob, token_id in zip(top_k_probs, top_k_ids):
        token_str = tokenizer.decode([token_id])

        # Check if word-start
        if is_word_start_token(token_str):
            word = decode_to_word(token_str)
            word_class = classify_candidate(word)

            mass[word_class] += prob.item()

            if debug and len(word_start_candidates) < 20:
                word_start_candidates.append({
                    'token': repr(token_str),
                    'word': word,
                    'class': word_class,
                    'prob': prob.item()
                })
        else:
            mass['non_wordstart'] += prob.item()

    # Compute residual (mass not in top-k)
    mass['residual'] = 1.0 - sum(mass.values())

    result = {
        'cue_word_index': cue_position,
        'mass': mass,
    }

    if debug:
        result['context'] = context[-60:] if len(context) > 60 else context
        result['top_candidates'] = word_start_candidates

    return result

print("✓ Analysis function defined")

## Step 9: Run Analysis on First 5 Stimuli (Preview)

In [ ]:
print("=" * 80)
print("MORPHOSYNTACTIC CONSTRAINT AUDIT - PREVIEW (First 5 Stimuli)")
print("=" * 80)
print()

# Run on first cue type only for preview
cue_type = 'infinitival_to'
spec = CUE_SPECS[cue_type]

print(f"CUE TYPE: {spec['description']}")
print("=" * 80)
print()

for stim_idx in range(min(5, len(stimuli))):
    stim = stimuli[stim_idx]
    
    print(f"\nSTIMULUS {stim_idx + 1}:")
    print("-" * 80)
    
    for condition in ['sentence', 'jabberwocky_matched', 'scrambled_jabberwocky']:
        text = stim[condition]
        
        # Try first cue
        cue_word = spec['cues'][0]
        result = analyze_cue_position(model, tokenizer, text, cue_word, top_k=5000, debug=True)
        
        if result is not None:
            print(f"\n{condition.upper()}:")
            print(f"  Context: ...{result['context']}")
            print(f"  Mass distribution:")
            for class_name in ['verb', 'noun', 'function', 'other_open', 'punct']:
                print(f"    {class_name:15s}: {result['mass'][class_name]*100:5.1f}%")
            print(f"  Top-10 word-start candidates:")
            for cand in result.get('top_candidates', [])[:10]:
                print(f"    {cand['token']:20s} [{cand['class']:12s}] {cand['prob']*100:5.1f}%")
        else:
            print(f"\n{condition.upper()}: (No '{cue_word}' found)")

print("\n" + "=" * 80)
print("Preview complete. Run full analysis in next cell.")
print("=" * 80)

## Step 10: Run Full Analysis (All Cue Types, All Stimuli)

In [ ]:
print("Running full analysis on all cue types and stimuli...")
print("(This may take 5-10 minutes)\n")

all_results = []

# For each cue type
for cue_type, spec in CUE_SPECS.items():
    print(f"\n{'='*80}")
    print(f"Processing: {spec['description']}")
    print(f"{'='*80}")
    
    cue_results = defaultdict(list)
    
    # Process each stimulus
    for stim_idx, stim in enumerate(stimuli):
        if (stim_idx + 1) % 10 == 0:
            print(f"  Processing stimulus {stim_idx + 1}/{len(stimuli)}...")
        
        for condition in ['sentence', 'jabberwocky_matched', 'scrambled_jabberwocky']:
            text = stim[condition]
            
            # Try each cue in this spec
            for cue_word in spec['cues']:
                result = analyze_cue_position(
                    model, tokenizer, text, cue_word,
                    top_k=5000, debug=False
                )
                
                if result is not None:
                    # Store result
                    result_record = {
                        'model': 'gpt2',
                        'cue_type': cue_type,
                        'condition': condition,
                        'stimulus_id': stim_idx,
                        'cue_word': cue_word,
                        **result
                    }
                    
                    all_results.append(result_record)
                    cue_results[condition].append(result['mass'])
    
    # Summary for this cue type
    print(f"\nSUMMARY: {spec['description']}")
    print("-" * 80)
    
    for condition in ['sentence', 'jabberwocky_matched', 'scrambled_jabberwocky']:
        if cue_results[condition]:
            masses = cue_results[condition]
            n = len(masses)
            
            mean_mass = {
                class_name: np.mean([m[class_name] for m in masses])
                for class_name in ['verb', 'noun', 'function', 'other_open', 'punct']
            }
            
            print(f"  {condition:30s} (n={n}):")
            for class_name in ['verb', 'noun', 'function', 'other_open', 'punct']:
                print(f"    {class_name:15s}: {mean_mass[class_name]*100:5.1f}%")

print("\n" + "=" * 80)
print("✓ Full analysis complete!")
print("=" * 80)

## Step 11: Save and Download Results

In [ ]:
# Save to JSON
output_file = 'morphosyntax_audit_results.json'

with open(output_file, 'w') as f:
    json.dump(all_results, f, indent=2)

print(f"✓ Results saved to {output_file}")
print(f"  Total records: {len(all_results)}")

# Download
from google.colab import files
files.download(output_file)
print("✓ Download started!")

## Step 12: Aggregate Summary Statistics

In [ ]:
print("=" * 80)
print("AGGREGATE SUMMARY: All Cue Types")
print("=" * 80)
print()

# Group by cue type and condition
for cue_type, spec in CUE_SPECS.items():
    print(f"\n{spec['description']}")
    print("-" * 80)
    
    # Filter results for this cue type
    cue_data = [r for r in all_results if r['cue_type'] == cue_type]
    
    if not cue_data:
        print("  (No data)")
        continue
    
    for condition in ['sentence', 'jabberwocky_matched', 'scrambled_jabberwocky']:
        condition_data = [r for r in cue_data if r['condition'] == condition]
        
        if condition_data:
            n = len(condition_data)
            
            # Compute mean and SEM for each class
            stats = {}
            for class_name in ['verb', 'noun', 'function', 'other_open']:
                values = [r['mass'][class_name] for r in condition_data]
                mean = np.mean(values)
                sem = np.std(values) / np.sqrt(n)
                stats[class_name] = (mean, sem)
            
            print(f"\n  {condition.upper()} (n={n}):")
            for class_name in ['verb', 'noun', 'function', 'other_open']:
                mean, sem = stats[class_name]
                print(f"    {class_name:15s}: {mean*100:5.1f}% ± {sem*100:4.2f}%")
    
    # Key contrasts
    sent_verb = np.mean([r['mass']['verb'] for r in cue_data if r['condition'] == 'sentence'])
    jab_verb = np.mean([r['mass']['verb'] for r in cue_data if r['condition'] == 'jabberwocky_matched'])
    scr_verb = np.mean([r['mass']['verb'] for r in cue_data if r['condition'] == 'scrambled_jabberwocky'])
    
    print(f"\n  KEY CONTRASTS:")
    print(f"    Sentence - Scrambled:     {(sent_verb - scr_verb)*100:+.1f}%")
    print(f"    Jabberwocky - Scrambled:  {(jab_verb - scr_verb)*100:+.1f}%")

print("\n" + "=" * 80)

## Interpretation

### What We're Testing

Does syntactic structure shift probability mass toward grammatically-appropriate lexical classes?

### Expected Patterns

1. **After "to" and modals**: 
   - High VERB mass in Sentence & Jabberwocky
   - Lower VERB mass in Scrambled
   - → Structure drives verb-slot selectivity

2. **After prepositions**:
   - Higher FUNCTION + NOUN mass in Sentence & Jabberwocky
   - Lower in Scrambled
   - → Structure drives NP-start selectivity

### What This Tells Us

**If we see these patterns**: Entropy reduction corresponds to category-level constraint at diagnostic morphosyntactic cues. The model's narrowed continuation space reflects grammatical structure, not just lexical statistics.

**For your paper**: 
> "After infinitival *to*, probability mass on verb continuations increased by X% in Jabberwocky relative to Scrambled, demonstrating that morphosyntactic cues—independent of lexical semantics—constrain the model's predictions toward grammatically-appropriate word classes."